In [44]:

from webbrowser import get
import requests
import base64
import urllib
import openai 
import pandas as pd
from urllib.parse import urlparse
import os
from dotenv import load_dotenv 
import json
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


In [49]:
readme = ""
summary = {}
with open("../sample_repos/catfact_readme.md","r") as file:
    readme = file.read()

with open("../generated_summaries/summary.json","r") as file:
    summary = json.load(file)


## Highlevel Summarization Tests

Using summaries, generate intro/getting started documentation

In [24]:
highlevel_prompt = f"""You are a markdown documentation planner. 
Please plan out a set of markdown files that will document the given code base. You should return the following:

1. The name of the .md file
2. A description of what the .md file will be about
3. For each .md file, list the code files that will be used to generate it 

Here is a dict of filepath:file summary for each file
{summary}

This documentation should be planned to help a developer understand the codebase, and high level functionality
"""

In [25]:
highlevel_completion = openai.Completion.create(
                        model="text-davinci-003",
                        prompt = highlevel_prompt,
                        temperature = 0,
                        max_tokens = 1000
            ).choices[0].text

highlevel_completion

"\n1. Introduction.md\n    - Description: This file will provide an introduction to the codebase, including a high level overview of the application and its purpose.\n    - Code Files: None\n\n2. Models.md\n    - Description: This file will provide an overview of the models used in the codebase, including a description of each model and its purpose.\n    - Code Files: app/models/api-log.js, app/models/fact.js, app/models/message.js, app/models/recipient.js, app/models/unsubscribe-date.js, app/models/user.js, app/models/verification-code.js\n\n3. Routes.md\n    - Description: This file will provide an overview of the routes used in the codebase, including a description of each route and its purpose.\n    - Code Files: app/routes/auth.routes.js, app/routes/catbot.routes.js, app/routes/console.routes.js, app/routes/contact.routes.js, app/routes/fact.routes.js, app/routes/index.js, app/routes/recipient.routes.js, app/routes/user.routes.js, app/routes/webhook.routes.js\n\n4. Authentication.

In [27]:
with open("../generated_docs/ai_planning.txt","w") as file:
    file.write(highlevel_completion)

FileNotFoundError: [Errno 2] No such file or directory: 'generated_docs/ai_planning.txt'

In [26]:
 Introduction.md\n    - Description: This file will provide an introduction to the codebase, including a high level overview of the application and its purpose.\n    - Code Files: None\n\n2. Models.md\n    - Description: This file will provide an overview of the models used in the codebase, including a description of each model and its purpose.\n    - Code Files: app/models/api-log.js, app/models/fact.js, app/models/message.js, app/models/recipient.js, app/models/unsubscribe-date.js, app/models/user.js, app/models/verification-code.js\n\n3. Routes.md\n    - Description: This file will provide an overview of the routes used in the codebase, including a description of each route and its purpose.\n    - Code Files: app/routes/auth.routes.js, app/routes/catbot.routes.js, app/routes/console.routes.js, app/routes/contact.routes.js, app/routes/fact.routes.js, app/routes/index.js, app/routes/recipient.routes.js, app/routes/user.routes.js, app/routes/webhook.routes.js\n\n4. Authentication.md\n    - Description: This file will provide an overview of the authentication process used in the codebase, including a description of the authentication middleware and the Google OAuth process.\n    - Code Files: app/routes/auth.routes.js\n\n5. Catbot.md\n    - Description: This file will provide an overview of the Catbot application, including a description of the daily facts, messages, and authentication routes.\n    - Code Files: app/routes/catbot.routes.js\n\n6. Console.md\n    - Description: This file will provide an overview of the console routes, including a description of the data that is returned and the authentication and admin middleware used.\n    - Code Files: app/routes/console.routes.js\n\n7. Contact.md\n    - Description: This file will provide an overview of the contact page, including a description of the Google People API and the filtering process used to get contacts with phone numbers.\n    - Code Files: app/routes/contact.routes.js\n\n8. Fact.md\n    - Description: This file will provide an overview of the Fact model, including a description of the routes used to get facts, submit facts, and get a random fact.\n    - Code Files: app/routes/fact.routes.js\n\n9. Recipient.md\n    - Description: This file will provide an overview of the Recipient model, including a description of the routes used to get, add, restore, and delete recipients, as well as get a recipient's conversation.\n    - Code Files: app/routes/recipient.routes.js\n\n10. User.md\n    - Description: This file will provide an overview of the user routes, including a description of the routes used to get the user's information, delete the user's account, verify the user's phone number, and update the user's phone number.\n    - Code Files: app/routes/user.routes.js\n\n11. Webhook.md\n    - Description: This file will provide an overview of the webhook route for the api.ai service, including a description of the requests it handles and the operations it performs.\n    - Code Files: app/routes/webhook.routes.js

SyntaxError: unexpected character after line continuation character (3669033536.py, line 1)

## Endpoint Generation

In [50]:
code_index = {}

with open("../cache/code_cache.json","r") as file:
    code_index = json.load(file)["cached_index"]

fact_endpoint = code_index["app/routes/fact.routes.js"]
fact_model = code_index["app/models/fact.js"]
user_model = code_index["app/models/user.js"]

In [52]:
code_index["app/models/user.js"]

"const mongoose = require('mongoose');\nconst Schema = mongoose.Schema;\nconst crypto = require('crypto');\nconst mongooseDelete = require('mongoose-delete');\n\nconst keys = require.main.require('./app/config/keys');\nconst strings = require.main.require('./app/config/strings');\n\n// Make email and phone docs unique except for those that are flagged as deleted\nconst uniquePartialIndex = {\n    unique: true,\n    partialFilterExpression: {\n        deleted: false\n    }\n};\n\nconst UserSchema = new Schema({\n    name: {\n        first:  {type: String, required: true},\n        last:   {type: String, required: true}\n    },\n    email:      {type: String},\n    phone:      {type: String},\n    photo:      {type: String, default: strings.userPhotoUrl},\n    google: {\n        id:           {type: String},\n        accessToken:  {type: String},\n        refreshToken: {type: String}\n    },\n    isAdmin: {type: Boolean, default: false},\n    ip: String\n}, {\n    timestamps: true\n});\n

In [12]:
prompt = f"""
Please write markdown docs for the endpoint(s) described in the following code. Format like so:

# Endpoint_Category Endpoints

## Endpoint1 Name

```
GET /endpoint1/path
```
Description:

Query Parameters: 

Example Reponse: 

## Endpoint2 Name

```
GET /endpoint2/path
```
Description:

Query Parameters: 

Example Response:*

endpoint: {fact_endpoint}
data model: {fact_model}
"""

In [13]:
endpoint_completion = openai.Completion.create(
                        model="code-davinci-002",
                        prompt = prompt,
                        temperature = 0,
                        max_tokens = 2048
            ).choices[0].text

md_text = endpoint_completion
with open("../generated_docs/nb_docs.md","w") as file:
    file.write(md_text)

In [20]:
endpoint_indexer_prompt = """
You are a api documentation planner
You will be given a code file which defines api endpoints. You will plan
a markdown file which documents these endpoints

1. Name the markdown file - be specific
2. Parse the file into a list of endpoints
3. For each endpoint, do the following:
    - name the endpoint
    - Give the string indices this endpoint is in between within the file.

        - example: "router.get(" is on line X, and the closing parenthesis ")" is on line Y
        return {"start_index":X, "end_index":Y}, 
    - Give the filepath to the data model this endpoint uses in its response

json output format:

{
    "md_file": "[placeholder].md",
    "endpoints":[
        {
            "name": "endpoint name",
            "line_indices": {"start_index":int, "end_index":int}, 
            "response_model": "data_model file name from directory root"
        }
        ...
    ]
}

code:

"""

In [21]:
endpoint_indexer_completion = openai.Completion.create(
                        model="text-davinci-003",
                        prompt = endpoint_indexer_prompt+fact_endpoint,
                        temperature = 0,
                        max_tokens = 2048
            ).choices[0].text

md_text = endpoint_indexer_completion
with open("../generated_docs/endpoint_index.md","w") as file:
    file.write(md_text)

In [7]:
endpoint_index = {
    "md_file": "api_endpoints.md",
    "endpoints":[
        {
            "name": "/",
            "line_indices": {"start_index": 8, "end_index": 16}, 
            "response_model": "./app/models/fact"
        },
        {
            "name": "/me",
            "line_indices": {"start_index": 18, "end_index": 28}, 
            "response_model": "./app/models/fact"
        },
        {
            "name": "/random",
            "line_indices": {"start_index": 30, "end_index": 43}, 
            "response_model": "./app/models/fact"
        },
        {
            "name": "/:factID",
            "line_indices": {"start_index": 45, "end_index": 54}, 
            "response_model": "./app/models/fact"
        },
        {
            "name": "/",
            "line_indices": {"start_index": 56, "end_index": 68}, 
            "response_model": "./app/models/fact"
        }
    ]
}

In [10]:
fact_lines = fact_endpoint.split("\n")
output_list = []
for endpoint in endpoint_index["endpoints"]:
    line_indices = endpoint["line_indices"]
    print(endpoint["name"])
    ep = fact_lines[line_indices["start_index"]:line_indices["end_index"]]
    print('----------------------------------')
    for line in ep:
        print(line)

/
----------------------------------

router.get('/', async(req, res) => {
    try {
        const facts = await Fact.find().limit(5);
        return res.status(200).json(facts);
    } catch (err) {
        return res.status(err.status || 400).json(err);
    }
/me
----------------------------------
// Get submitted facts
router.get('/me', async(req, res) => {

    const animalType = req.query.animal_type ? req.query.animal_type.split(',') : ['cat'];

    try {
        const data = await Fact.find({
                user: req.user._id,
                type: { $in: animalType }
            })
/random
----------------------------------
            .limit(10)

        return res.status(200).json(data);
    } catch (err) {
        return res.status(400).json(err);
    }
});

// Get a random fact
router.get('/random', logApiRequest, async(req, res) => {

    const animalType = req.query.animal_type ? req.query.animal_type.split(',') : ['cat'];
    const amount = req.query.amount;
/:factID
---

In [12]:
with open("../sample_repos/facts.js","w") as file:
    file.write(fact_endpoint)

import re

pattern = re.compile(r"^(get|post|put|delete|patch|options|head)\(['\"]([^'\"]*)['\"],\s*(.*)\)$")


In [34]:
import re

# define the regex pattern
pattern = re.compile(r"router\.(get|post|put|delete)\('(.+?)',.*\)", re.DOTALL)

endpoint_list = []
match = False
start_idx = None
end_idx = None
paren_stack = []
paren_map = {
    ")":"("
}
# read in the file
with open("../sample_repos/facts.js", 'r') as file:
    # iterate over each line in the file
    for line_number, line in enumerate(file):
        # search for the pattern in the line
        if not match:
            match = pattern.search(line)
        # if a match is found, print the line number and the matched text
        if match:
            if start_idx == None:
                start_idx = line_number
                print(f"Match found on line {start_idx}: {match.group()}")
            else:
                for char in line:
                    if char == "(":
                        paren_stack.append(char)
                    elif char == ")":
                        if len(paren_stack)==0:
                            end_idx = line_number
                            endpoint_list.append((start_idx, end_idx))
                            print(f"endpoint form {start_idx,end_idx}")
                            match = False
                            start_idx = None
                        else:
                            paren_stack.pop(-1)



Match found on line 9: router.get('/', async(req, res)
endpoint form (9, 16)
Match found on line 19: router.get('/me', async(req, res)
endpoint form (19, 36)
Match found on line 39: router.get('/random', logApiRequest, async(req, res)
endpoint form (39, 54)
Match found on line 57: router.get('/:factID', logApiRequest, async(req, res)
endpoint form (57, 69)
Match found on line 72: router.post('/', isAuthenticated, async(req, res)
endpoint form (72, 109)


In [ ]:
randomfact = """
// Get a random fact
router.get('/random', logApiRequest, async(req, res) => {

    const animalType = req.query.animal_type ? req.query.animal_type.split(',') : ['cat'];
    const amount = req.query.amount;

    if (amount > 500) {
        return res.status(405).json({ message: 'Limited to 500 facts at a time' });
    }

    try {
        const facts = await Fact.getFact({ amount, animalType });
        return res.status(200).json(facts);
    } catch (err) {
        return res.status(err.status).json(err);
    }
});"""

p = f"""Produce markdown documentation for the following api endpoint. 

code: {randomfact}
file imports: {fact_model}

Format like so:

## Endpoint Name

```
GET /endpoint1/path
```
Description:

Query Parameters: 

Example Reponse: 

"""


ep_completion = openai.Completion.create(
                        model="text-davinci-003",
                        prompt = p,
                        temperature = 0,
                        max_tokens = 2048
            ).choices[0].text

ep_completion

In [48]:
randomfact = """
// Get a random fact
router.get('/random', logApiRequest, async(req, res) => {

    const animalType = req.query.animal_type ? req.query.animal_type.split(',') : ['cat'];
    const amount = req.query.amount;

    if (amount > 500) {
        return res.status(405).json({ message: 'Limited to 500 facts at a time' });
    }

    try {
        const facts = await Fact.getFact({ amount, animalType });
        return res.status(200).json(facts);
    } catch (err) {
        return res.status(err.status).json(err);
    }
});"""

filepath_prompt = f"""Give the filepath, starting with the root dir, for the data model this api endpoint uses in its response
You will be given endpoint code, and the file imports.

code: 
{randomfact}

file imports: 
const express = require('express');
const router = express.Router();

const strings = require.main.require('./app/config/strings.js');
const ( isAuthenticated, logApiRequest ) = require('../middleware');

const Fact = require.main.require('./app/models/fact');
const User = require.main.require('./app/models/user');


"""


path_completion = openai.Completion.create(
                        model="text-davinci-003",
                        prompt = filepath_prompt,
                        temperature = 0,
                        max_tokens = 100
            ).choices[0].text

path_completion

'Answer: /app/models/fact.js'

In [24]:
with open("../generated_docs/nb_docs.md","w") as file:
    file.write(ep_completion)

## Get a Random Fact\n\n```\nGET /random\n```\nDescription: This endpoint returns a random fact about an animal. \n\nQuery Parameters: \n- animal_type: A comma separated list of animal types to get facts about. Defaults to \'cat\'.\n- amount: The number of facts to return. Limited to 500 facts at a time.\n\nExample Response: \n```\n{\n    "user": "5f3f3f3f3f3f3f3f3f3f3f3f",\n    "text": "Cats have over 20 muscles that control their ears.",\n    "sendDate": "2020-09-09T00:00:00.000Z",\n    "type": "cat",\n    "status": {\n        "verified": true,\n        "feedback": "",\n        "sentCount": 0\n    }\n}\n```